# Pytorch Model
daniel.brooks@alumni.caltech.edu <br>
Feburary 24, 2018 <br>  

Next step: configure NN.

In [1]:
#Preprocessing imports.
import numpy as np
from sklearn import preprocessing

import draftsimtools as ds

In [2]:
#Torch imports.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#Load 1000 M19 drafts.
m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")
raw_drafts = ds.load_drafts("data/m19_1000drafts.csv")

In [4]:
#Fix commas in card names.
m19_set, raw_drafts = ds.fix_commas(m19_set, raw_drafts)

In [5]:
#Store rating information in a dictionary.
rating_dict = ds.create_rating_dict(m19_set)

In [6]:
#Process the draft data.
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.


In [7]:
#Create an M19 player. 
#b = ds.SGDBot(rating_dict)

In [8]:
#%%time
#Optimize rating parameters using stochastic gradient descent. (1 minute / 1000 drafts)
#for x in range(10):
#    b.sgd_optimization(drafts[0:25], 0.05)
#print("Done.")

In [9]:
#Write new ratings to file.
#b.write_rating_dict("sgd_05_linear.tsv")
#b.write_error("error.csv")

# PyTorch Preprocessing

In [10]:
#Save draft data as a tensor.
def create_le(cardnames):
    """Create label encoder for cardnames."""
    le = preprocessing.LabelEncoder()
    le.fit(cardnames)
    return le

def draft_to_matrix(cur_draft, le, pack_size=15):
    """Transform draft from cardname list to one hot encoding."""
    pick_list = [np.append(le.transform(cur_draft[i]), (pack_size-len(x))*[0]) \
                 for i, x in enumerate(cur_draft)]
    pick_matrix = np.int16(pick_list)
    return pick_matrix

def drafts_to_tensor(drafts, le, pack_size=15):
    """Create tensor of shape (num_drafts, 45, 15)."""
    pick_tensor_list = [draft_to_matrix(d, le) for d in drafts]
    pick_tensor = np.int16(pick_tensor_list)
    return pick_tensor

#Create drafts tensor.
le = create_le(m19_set["Name"].values)
drafts_tensor = drafts_to_tensor(drafts, le)

In [11]:
drafts_tensor.shape

(1000, 45, 15)

In [12]:
#Create torch dataset class.
from torch.utils.data.dataset import Dataset

#Drafts dataset class.
class DraftDataset(Dataset):
    """Defines a draft dataset in PyTorch."""
    
    def __init__(self, drafts_tensor, le):
        """Initialization."""
        self.drafts_tensor = drafts_tensor
        self.le = le
        self.cards_in_set = len(self.le.classes_)
        self.pack_size = int(self.drafts_tensor.shape[1]/3)
        
    def __getitem__(self, index):
        """Return a training example.
        """
        #Compute number of picks in a draft.
        draft_size = self.pack_size*3
        
        #Grab information on current draft.
        pick_num = index % draft_size #0-self.pack_size*3-1
        draft_num = int((index - pick_num)/draft_size)
        
        #Generate.
        x = self.create_x(pick_num, draft_num)
        y = self.create_y(pick_num, draft_num)
        return x, y
    
    def create_x(self, pick_num, draft_num):
        """Generate x, input. 
        Column 1: collection vector
                  x[i]=n -> collection has n copies of card i
        Column 2: pack vector
                  0 -> card not in pack
                  1 -> card in pack
        Efficiency optimization possible. Iterative adds to numpy array.
        """
        #Initialize collection / cards in pack vector.
        x = np.zeros([self.cards_in_set, 2], dtype = "int16")
        
        #Fill in collection vector excluding current pick.
        for n in self.drafts_tensor[draft_num, :pick_num, 0]:
            x[n, 0] += 1
            
        #Fill in pack vector.
        cards_in_pack =  self.pack_size - pick_num%self.pack_size #Cards in current pack.
        for n in self.drafts_tensor[draft_num, pick_num, :cards_in_pack]:
            x[n, 1] = 1
        
        #Convert to Torch tensor. 
        #print("TEST: LEAVING OUT PACK VECTOR.")
        #x = torch.Tensor(x[:,0])
        x = torch.Tensor(x)
        return x
    
    def create_y(self, pick_num, draft_num, not_in_pack=0.5):
        """Generate y, a target pick vector.
        Picked card is assigned a value of 1.
        Other cards are assigned a value of 0.
        """
        #Initialize target vector.
        #y = np.array([0] * self.cards_in_set)
        y = np.zeros([self.cards_in_set, 1], dtype = "int16")
            
        #Add picked card.
        y[self.drafts_tensor[draft_num, pick_num, 0]] = 1
        y = torch.Tensor(y)
        return y
    
    def __len__(self):
        return len(self.drafts_tensor)
    
#Create a draft dataset.
d = DraftDataset(drafts_tensor, le)

In [13]:
#Implement NN.
class Net(nn.Module):
    
    def __init__(self, ss):
        """Placeholder NN. Currently does nothing.
        
        param ss: number of cards in set
        """
        super(Net, self).__init__()
        #self.fc1 = nn.Linear(ss,ss)
        #self.fc2 = nn.Linear(ss,ss)
        #self.sm = nn.Softmax(dim=0)
        
        H = 10
        D_in = 2
        D_out = ss
        
        #Placeholder. 
        #Method believes an example is x[:,i,:] -> y[:,i,:]
        #Need an example to be x[0,:,:] -> y[0,:,:]
        #x -> (4, 285, 2)
        #y -> (4, 285, 1)
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        
        #x dimensions are (batch, cards_set, 2)
        #x0 = x[:, 0] #Collection
        #x1 = x[:, :, 1] #Cards in pack
        
        #print(x.shape)
        
        #xt = np.transpose(x)
        #x0 = xt[0,:]
        
        #Desired forward prop function.
        #out = self.fc1(x0)
        #out = F.relu(self.fc1(x))
        #out = F.relu(self.fc2(out))
        #out = out * x1 #Constrain cards based on what is in pack.
        #out = self.sm(out)
        #out = x
        h_relu = self.linear1(x)
        y_pred = self.linear2(h_relu)
        return y_pred

#Create NN.
net = Net(len(m19_set))
print(net)

Net(
  (linear1): Linear(in_features=2, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=285, bias=True)
)


In [14]:
###############################
# Define training parameters. #
###############################

#Define training set. Batchsize must be 1.
trainset = d
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

#Loss function.
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()

#Define optimizer. 
optimizer = optim.SGD(net.parameters(), lr=0.5, momentum=0.9)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
# Train network.

#Set up pulled from pytorch tutorial.
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-download-beginner-blitz-cifar10-tutorial-py

NUM_EPOCH = 20
num_draft = len(trainloader)

for epoch in range(NUM_EPOCH):
    
    #Loop over x,y for each dataset.
    running_loss = 0
    for i, data in enumerate(trainloader):
        
        #Get the inputs. Keeps batch size.
        inputs, labels = data
        #print(inputs.shape, labels.shape)
        #inputs = data[0][0,:,:]
        #labels = data[1][0,:,:]
        #print(inputs.shape, labels.shape)
                
        #Zero parameter gradients between batches.
        optimizer.zero_grad()
        
        #Perform training.
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #Print loss data.
        running_loss += loss.item()
        if i % num_draft == num_draft-1:
            print('Epoch %d, Average Loss: %.6f' % (epoch+1, running_loss/num_draft))
            running_loss = 0.0
            
print("Finished Training")

Epoch 1, Average Loss: 0.007219
Epoch 2, Average Loss: 0.003469
Epoch 3, Average Loss: 0.003458
Epoch 4, Average Loss: 0.003450
Epoch 5, Average Loss: 0.003441
Epoch 6, Average Loss: 0.003430
Epoch 7, Average Loss: 0.003418
Epoch 8, Average Loss: 0.003403
Epoch 9, Average Loss: 0.003387
Epoch 10, Average Loss: 0.003369
Epoch 11, Average Loss: 0.003349
Epoch 12, Average Loss: 0.003328
Epoch 13, Average Loss: 0.003306
Epoch 14, Average Loss: 0.003284
Epoch 15, Average Loss: 0.003261
Epoch 16, Average Loss: 0.003239
Epoch 17, Average Loss: 0.003218
Epoch 18, Average Loss: 0.003198
Epoch 19, Average Loss: 0.003179
Epoch 20, Average Loss: 0.003163
Finished Training
